In [6]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import load_model
from keras.backend import expand_dims


os.chdir('/home/tdeng/PBMC/tree-cnn/')
layer_models = []
# load models
for i in range(1, 5):
    path = os.path.join('train', 'layer' + str(i), 'normal_cnn.h5')
    layer_models.append(load_model(path))

    
X_test = np.load('test/X_test.npy')
y_test = np.load('test/y_test.npy')
X_test = expand_dims(X_test, axis=-1).numpy()
print(X_test.shape)


layer1_result = layer_models[0].predict(X_test[:1000])
layer2_result = layer_models[1].predict(X_test[:1000])
layer3_result = layer_models[2].predict(X_test[:1000])
layer4_result = layer_models[3].predict(X_test[:1000])

pred_result = []
for i in range(1000):
    LP_cell_prob = layer1_result[i, 0]
    MP_cell_prob = layer1_result[i, 1]
    B_cell_prob = layer2_result[i, 0]
    T_cell_progenitor_prob = layer2_result[i, 1]
    CD56_NK_prob = layer2_result[i, 2]
    Naive_cytotoxic_T_prob = layer3_result[i, 0]
    Naive_T_prob = layer3_result[i, 1]
    regulatory_T_prob = layer4_result[i, 0]
    CD4_T_helper_prob = layer4_result[i, 1]
    memory_T_prob = layer4_result[i, 2]
    
    true_B_cell = LP_cell_prob * B_cell_prob
    true_CD4_T_helper = LP_cell_prob * T_cell_progenitor_prob * Naive_T_prob * CD4_T_helper_prob
    true_CD14_monocytes = MP_cell_prob
    true_CD56_NK = LP_cell_prob * CD56_NK_prob
    true_memory_T = LP_cell_prob * T_cell_progenitor_prob * Naive_T_prob * memory_T_prob
    true_naive_T = LP_cell_prob * T_cell_progenitor_prob * Naive_T_prob
    true_naive_cytotoxic_T = LP_cell_prob * T_cell_progenitor_prob * Naive_cytotoxic_T_prob
    true_regulatory_T = LP_cell_prob * T_cell_progenitor_prob * Naive_T_prob * regulatory_T_prob

    pred_result.append(np.argmax([true_B_cell, true_CD4_T_helper, true_CD14_monocytes, true_CD56_NK,
                    true_memory_T, true_naive_cytotoxic_T, true_regulatory_T]))

result = np.array(pred_result)
print(result)
print(y_test[:1000])
print(accuracy_score(y_test[:1000], result))


    


(18393, 32738, 1)
[6 1 5 1 1 5 5 1 5 3 6 3 5 5 0 6 4 4 6 3 0 6 6 4 3 6 0 4 3 1 1 6 0 0 0 3 5
 4 4 5 5 3 1 3 1 3 6 1 5 0 6 6 3 5 3 6 5 1 5 6 5 6 5 6 4 0 1 0 5 1 6 5 5 1
 0 6 5 4 4 3 5 2 5 5 5 3 6 4 3 3 4 3 3 4 4 0 0 3 5 0 6 6 4 1 4 0 0 1 0 6 3
 3 0 1 6 3 5 1 5 1 1 1 6 1 3 0 5 5 1 5 0 5 5 3 3 6 1 5 1 3 4 6 1 1 1 6 6 1
 1 0 5 6 1 5 5 5 3 4 3 6 5 4 3 6 6 4 3 4 3 5 3 6 0 6 1 4 1 6 3 1 1 1 3 5 3
 6 5 4 3 4 4 6 0 5 1 1 1 1 1 4 5 2 6 4 6 1 5 3 0 3 1 3 1 3 1 6 0 3 4 1 1 4
 0 2 4 5 3 1 0 0 5 3 4 3 0 5 6 1 6 5 1 5 4 5 6 3 4 1 1 4 3 1 1 4 6 5 3 1 5
 5 5 1 0 4 6 4 4 5 1 3 4 6 4 5 3 4 0 5 5 1 3 1 0 5 1 0 1 5 6 4 1 4 5 4 0 1
 1 1 1 5 4 6 1 0 1 6 1 5 6 4 4 6 1 3 6 6 6 6 3 6 6 4 1 0 6 6 4 4 6 0 6 1 3
 1 1 0 1 3 2 0 1 6 6 0 6 5 2 0 6 4 1 0 5 6 5 5 4 3 5 3 4 1 5 3 4 5 3 5 4 5
 5 1 4 5 1 3 3 0 6 6 1 3 5 3 1 4 6 3 3 1 1 0 4 5 6 6 4 1 5 5 0 5 1 0 4 6 3
 5 0 5 0 0 1 6 0 0 5 4 5 0 4 4 3 5 5 1 3 0 5 5 4 1 1 6 6 6 5 0 6 6 0 4 6 0
 6 0 3 6 6 0 1 0 1 1 6 4 5 3 4 6 0 6 6 6 3 4 1 0 3 5 1 1 4 5 4 0 3 5 0 6 6
 1 1 4 